In [1]:
import warnings

warnings.filterwarnings("ignore", message=r".*distutils.*")

In [2]:
import boto3
import pandas as pd
from io import StringIO

# Инициализация клиента
s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="some_aws_access_key_id",
    aws_secret_access_key="some_aws_secret_access_key",
)

# Считывание данных
obj = s3.get_object(Bucket="datasets", Key="kinopoisk_train.csv")
data = obj["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(data))

In [3]:
df

,review,sentiment
0,"Смотрел фильм несколько раз, с первого раза вы...",neu
1,Я даже не знаю с чего начать... Сериал этот я ...,neg
2,Одна вещь порождает следующую. Человек имеет с...,pos
3,"«Журнал 64», как и вся серия адаптации книг эт...",neg
4,"Однажды утром просматривая почту, Дон Джонстон...",pos
...,...,...
19746,Перед нами одна из многочисленных короткометра...,pos
19747,Очень редко смотрю современные комедии - как н...,pos
19748,При просмотре третьей части меня никак не поки...,neg
19749,Как всегда поражаюсь удивительной сдержанности...,pos


In [4]:
%env MLFLOW_TRACKING_URI=http://localhost:5000
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000

env: MLFLOW_TRACKING_URI=http://localhost:5000
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000


In [5]:
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    df["review"], df["sentiment"], test_size=0.2
)

# Векторизация
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели
clf = LogisticRegression(solver="lbfgs", max_iter=1000)
clf.fit(X_train_vec, y_train)

# Предсказание
y_pred = clf.predict(X_test_vec)

# Настройка клиента boto3
boto3.setup_default_session(
    aws_access_key_id="some_aws_access_key_id",
    aws_secret_access_key="some_aws_secret_access_key",
    region_name="us-west-1",
)

# Логирование в MLflow
with mlflow.start_run() as run:
    # Логирование параметров и метрик
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))

    # Логирование модели
    mlflow.sklearn.log_model(clf, "model", registered_model_name="SomeModel")

Successfully registered model 'SomeModel'.
2024/03/26 16:40:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: SomeModel, version 1
Created version '1' of model 'SomeModel'.


In [6]:
accuracy_score(y_test, y_pred)

0.7625917489243229